In [328]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score

In [329]:
#read in df with dummies
former_final_df = pd.read_csv('../data/final_df.csv')

C:\Users\Conrad\AppData\Local\Temp\ipykernel_24404\279734686.py:2: DtypeWarning: Columns (5,6,9,10,12,13,14,15,16,17,18,19,21,23,24,25,26,27) have mixed types. Specify dtype option on import or set low_memory=False.
  former_final_df = pd.read_csv('../data/final_df.csv')


In [330]:
# Set view options
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [331]:
former_final_df = former_final_df[former_final_df['Aftertreatment System'].notna()]

In [332]:
almost_final_df = former_final_df.drop(former_final_df.loc[:,'AcceleratorPedal':'TurboBoostPressure'], axis = 1)
final_df = almost_final_df.drop(['EventTimeStamp', 'ESS_Id', 'fmi'], axis = 1)
final_df

,EquipmentID,spn,Aftertreatment System,Ambient Air Temperature,Anti-Lock Braking (ABS) Controller,Auxiliary Input/Output,Auxiliary Temperature Sensor Input Circuit,Barometric Pressure Sensor,Battery,Crankcase,DAS Module to ECM Communication,ECU Power Output Supply Voltage,Electric Lift Pump,Electrical Charging System Voltage,Electronic Fuel Injection Control Valve,Electronic Throttle Control,Engine Air Filter,Engine Brake,Engine Charge Air Cooler Outlet Temperature,Engine Control Module,Engine Coolant,Engine Cooling System,Engine Diesel Particulate Filter,Engine Exhaust Back Pressure,Engine Fuel,Engine Injector Metering Rail,Engine Intake Air Heater,Engine Misfire,Engine Oil,Engine Position Sensor,Engine Protection Torque Full Derate,Engine Protection Torque Partial Derate,Engine Speed,Engine Starter Motor Relay,Engine Total Idle Hours,Exhaust Gas,Exhaust Gas Recirculation System,Fan Control,Fuel Control Module,High Pressure Common Rail Fuel Pressure Relief Valve,Indicator Lamp,Injector Metering Rail,Injector Solenoid,Intake Manifold,Intake Throttle,J1939 Network,Real Time Clock,Sensor Supply,Steering Wheel Angle Sensor,Total Vehicle Distance,Transmission Communications Failure Idle Torque Limit,Transmission Oil,Turbocharger,Unknown,Variable Geometry Turbocharger,Vehicle Identification Number,Wheel-Based Vehicle Speed
1,301,596,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,301,3226,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,301,639,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,301,596,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
5,301,639,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
554847,2377,1231,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
554848,2377,1231,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
554850,2377,111,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
554851,2377,111,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [333]:
#Get list of unique EquipmentIDs
eq_ids = final_df.drop_duplicates(subset='EquipmentID')
eq_ids = eq_ids[['EquipmentID']]
eq_ids.shape[0]

1026

In [334]:
#create a column of random numbers from 0-1
np.random.seed(321)

eq_ids['random'] = (np.random.randint(0, 10000, eq_ids.shape[0]))/10000

eq_ids

,EquipmentID,random
1,301,0.6682
433,302,0.4220
2050,303,0.5929
2787,304,0.9617
4024,305,0.8360
...,...,...
554816,2371,0.1721
554821,2374,0.3518
554834,2375,0.0887
554839,2376,0.5903


In [335]:
#create column 'modeling' labeling ~35% of the data for training, ~30% for validation, and ~35% for testing 
eq_ids['modeling'] = np.where(((eq_ids.random <= 0.35)), 'training', np.where(((eq_ids.random <= 0.65)), 'validation', 'testing'))

In [336]:
#merge modeling column onto original dataframe
final_df = final_df.sort_values(by='EquipmentID')
eq_ids = eq_ids.sort_values(by='EquipmentID')

final_df = final_df.merge(eq_ids, on='EquipmentID')

In [337]:
#created training dataframe
df_training = final_df[final_df['modeling']=='training']
df_training = df_training.drop(columns=['modeling', 'random'])
df_training

,EquipmentID,spn,Aftertreatment System,Ambient Air Temperature,Anti-Lock Braking (ABS) Controller,Auxiliary Input/Output,Auxiliary Temperature Sensor Input Circuit,Barometric Pressure Sensor,Battery,Crankcase,DAS Module to ECM Communication,ECU Power Output Supply Voltage,Electric Lift Pump,Electrical Charging System Voltage,Electronic Fuel Injection Control Valve,Electronic Throttle Control,Engine Air Filter,Engine Brake,Engine Charge Air Cooler Outlet Temperature,Engine Control Module,Engine Coolant,Engine Cooling System,Engine Diesel Particulate Filter,Engine Exhaust Back Pressure,Engine Fuel,Engine Injector Metering Rail,Engine Intake Air Heater,Engine Misfire,Engine Oil,Engine Position Sensor,Engine Protection Torque Full Derate,Engine Protection Torque Partial Derate,Engine Speed,Engine Starter Motor Relay,Engine Total Idle Hours,Exhaust Gas,Exhaust Gas Recirculation System,Fan Control,Fuel Control Module,High Pressure Common Rail Fuel Pressure Relief Valve,Indicator Lamp,Injector Metering Rail,Injector Solenoid,Intake Manifold,Intake Throttle,J1939 Network,Real Time Clock,Sensor Supply,Steering Wheel Angle Sensor,Total Vehicle Distance,Transmission Communications Failure Idle Torque Limit,Transmission Oil,Turbocharger,Unknown,Variable Geometry Turbocharger,Vehicle Identification Number,Wheel-Based Vehicle Speed
6490,306,829,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6491,306,829,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6492,306,96,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
6493,306,829,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
6494,306,639,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
545692,2371,111,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
545704,2375,2863,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
545705,2375,37,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0
545706,2375,37,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


## Over/Under Sampling

In [338]:
#relabeling spn codes to either partial, full, or none
df_sampling = df_training


df_sampling['spn'] = df_sampling['spn'].mask(df_sampling['spn'] == 1569, 'partial')
df_sampling['spn'] = df_sampling['spn'].mask(df_sampling['spn'] == 5246, 'full')
df_sampling['spn'] = df_sampling['spn'].mask((df_sampling['spn'] != 'partial') & (df_sampling['spn'] != 'full'), 'none')
df_sampling['spn'].unique()

array(['none', 'full', 'partial'], dtype=object)

In [ ]:
#dataframe of just partial and full derates for SMOTE oversampling
df_sampling_derates = df_sampling[(df_sampling['spn']=='full') | (df_sampling['spn']=='partial')]

### SMOTE round 1

In [340]:
#set up SMOTE 
oversampler = SMOTE(k_neighbors=5, random_state=321)

In [341]:
#prep for df_sampling_derates to go through SMOTE
features = df_sampling_derates.iloc[:,2:].columns.tolist()

target =['spn']

X_train = df_sampling_derates[features]
y_train = df_sampling_derates[target]

In [342]:
X_smote, y_smote = oversampler.fit_resample(X_train, y_train)

In [343]:
#combining the resulting resampled data back into one dataframe 
Smote_data = pd.concat([y_smote, X_smote], axis=1)
Smote_data.shape

(3090, 56)

### SMOTE round 2

In [344]:
#Making dataframe of 1/20th size of non derate rows.
#not 1/10 as planned because these resamplers are binary so I have to run partial and full derate oversampling separately
df_sampling_none = df_sampling[df_sampling['spn']=='none']
df_sampling_none_twentieth = df_sampling_none[0:975]

In [345]:
#creating dataframes for SMOTE round 2
df_sampling_partial_twentieth = pd.concat([Smote_data[Smote_data['spn']=='partial'], df_sampling_none_twentieth])
df_sampling_full_twentieth = pd.concat([Smote_data[Smote_data['spn']=='full'], df_sampling_none_twentieth])

In [346]:
oversampler_2 = SMOTE(k_neighbors=5, random_state=321)

features = df_sampling.iloc[:,2:].columns.tolist()

target =['spn']

X_train_partial = df_sampling_partial_twentieth[features]
y_train_partial = df_sampling_partial_twentieth[target]

X_train_full = df_sampling_full_twentieth[features]
y_train_full = df_sampling_full_twentieth[target]

In [347]:
X_smote_partial, y_smote_partial = oversampler_2.fit_resample(X_train_partial, y_train_partial)

X_smote_full, y_smote_full = oversampler_2.fit_resample(X_train_full, y_train_full)

In [348]:
#combining derates back together before attaching to original non derate rows for undersampling
smote_partial = pd.concat([X_smote_partial, y_smote_partial], axis=1)
smote_full = pd.concat([X_smote_full, y_smote_full], axis=1)
smote_final = pd.concat([smote_partial, smote_full])
smote_final = smote_final[smote_final['spn']!="none"]
smote_final.shape

(3090, 56)

### Undersampling

In [349]:
#combine original non-derate rows with new resampled derates
und_samp = pd.concat([smote_final, df_sampling_none])
#Rename full and partial to just derate 
und_samp['spn'] = und_samp['spn'].mask((und_samp['spn'] == 'partial') | (und_samp['spn'] == 'full'), 'derate')
und_samp = und_samp.reset_index().drop(columns='index')
und_samp.shape

(199048, 57)

In [350]:
#prep for undersampler
undersampler = RandomUnderSampler(random_state=321, sampling_strategy=1/2)

features = df_sampling.iloc[:,2:].columns.tolist()

target =['spn']

X_train_final = und_samp[features]
y_train_final = und_samp[target]

In [351]:
X_resampled, y_resampled = undersampler.fit_resample(X_train_final, y_train_final)

In [372]:
#resulting numbers
print(y_resampled.shape)

(9270, 1)


### Test model

In [364]:
logistic = LogisticRegression(max_iter=10000)

In [365]:
#logistic fit on resampled data
logreg = logistic.fit(X_resampled, y_resampled)

C:\Users\Conrad\anaconda3\envs\p310\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [369]:
#changing target variable classification in test df to match with training
df_test = final_df[final_df['modeling']=='testing']
df_test = df_test.drop(columns=['modeling','random'])
df_test['spn'] = df_test['spn'].mask((df_test['spn'] == 1569) | (und_samp['spn'] == 5246), 'derate')
df_test['spn'] = df_test['spn'].mask(df_test['spn'] != 'derate', 'none')
df_test

,EquipmentID,spn,Aftertreatment System,Ambient Air Temperature,Anti-Lock Braking (ABS) Controller,Auxiliary Input/Output,Auxiliary Temperature Sensor Input Circuit,Barometric Pressure Sensor,Battery,Crankcase,DAS Module to ECM Communication,ECU Power Output Supply Voltage,Electric Lift Pump,Electrical Charging System Voltage,Electronic Fuel Injection Control Valve,Electronic Throttle Control,Engine Air Filter,Engine Brake,Engine Charge Air Cooler Outlet Temperature,Engine Control Module,Engine Coolant,Engine Cooling System,Engine Diesel Particulate Filter,Engine Exhaust Back Pressure,Engine Fuel,Engine Injector Metering Rail,Engine Intake Air Heater,Engine Misfire,Engine Oil,Engine Position Sensor,Engine Protection Torque Full Derate,Engine Protection Torque Partial Derate,Engine Speed,Engine Starter Motor Relay,Engine Total Idle Hours,Exhaust Gas,Exhaust Gas Recirculation System,Fan Control,Fuel Control Module,High Pressure Common Rail Fuel Pressure Relief Valve,Indicator Lamp,Injector Metering Rail,Injector Solenoid,Intake Manifold,Intake Throttle,J1939 Network,Real Time Clock,Sensor Supply,Steering Wheel Angle Sensor,Total Vehicle Distance,Transmission Communications Failure Idle Torque Limit,Transmission Oil,Turbocharger,Unknown,Variable Geometry Turbocharger,Vehicle Identification Number,Wheel-Based Vehicle Speed
0,301,none,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,301,none,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13.0,0.0,0.0,0.0
2,301,none,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,14.0,0.0,0.0,0.0
3,301,none,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13.0,0.0,0.0,0.0
4,301,none,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
545715,2377,none,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
545716,2377,none,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
545717,2377,none,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
545718,2377,none,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0


In [370]:
X_test = df_test[features]
y_test = df_test[target]

In [371]:
y_pred = logreg.predict(X_test)

print(accuracy_score(y_test,y_pred))
print(f1_score(y_test, y_pred, average = 'weighted'))

0.9554580774533149
0.9702377036015868
